In [63]:
!pip install langchain langchain-groq langchain-cohere langgraph pymilvus requests

In [64]:
import os
from langchain_cohere import CohereEmbeddings
from langchain_community.vectorstores import Milvus
from langchain_groq import ChatGroq
from langgraph.graph import StateGraph, END
from langgraph.prebuilt import ToolNode
from langchain.tools import Tool
import requests

# --- API Keys ---
os.environ["GROQ_API_KEY"] = ""
os.environ["COHERE_API_KEY"] = ""

ZILLIZ_URI = ""
ZILLIZ_API_KEY = ""

COLLECTION_NAME = "rag_knowledge_base"

In [65]:
# --- Vector Store Setup ---
embeddings = CohereEmbeddings(model="embed-english-v3.0")

vector_store = Milvus(
    embedding_function=embeddings,
    connection_args={"uri": ZILLIZ_URI, "token": ZILLIZ_API_KEY},
    collection_name=COLLECTION_NAME,
)

# --- Define Tools ---

def search_vector_store(query: str) -> str:
    """Tool 1: Search vector store for factual/knowledge-based info."""
    try:
        results = vector_store.similarity_search(query, k=3)
        if not results:
            return "No relevant information found in the database."
        return "\n".join([doc.page_content for doc in results])
    except Exception as e:
        return f"Vector Store Error: {e}"

def get_weather(location: str) -> str:
    """Tool 2: Get weather info for a city using OpenWeatherMap API."""
    api_key = os.environ.get("OPENWEATHERMAP_API_KEY")
    if not api_key:
        return "OpenWeatherMap API key not found. Please add it to your environment variables."
    base_url = "http://api.openweathermap.org/data/2.5/weather?"
    complete_url = base_url + "appid=" + api_key + "&q=" + location + "&units=metric"
    try:
        response = requests.get(complete_url)
        data = response.json()
        if data["cod"] != "404":
            main_data = data["main"]
            current_temperature = main_data["temp"]
            weather_description = data["weather"][0]["description"]
            return f"The weather in {location} is {current_temperature}°C with {weather_description}."
        else:
            return f"City not found: {location}"
    except Exception as e:
        return f"Error fetching weather data: {e}"


# Wrap tools for LangGraph
tools = [
    Tool(
        name="VectorStoreSearch",
        func=search_vector_store,
        description="Use this to answer factual or technical queries about AI, ML, or general knowledge."
    ),
    Tool(
        name="WeatherAPI",
        func=get_weather,
        description="Use this for questions about weather, temperature, or climate in a city."
    )
]

In [66]:
from langchain_core.pydantic_v1 import BaseModel, Field

# Define the reasoning input/output schema
class AgentState(BaseModel):
    query: str
    reasoning: str = Field(default="")
    final_answer: str = Field(default="")
    tool_output: str = Field(default="")
    next_action: str = Field(default="reason")  # reason → act → observe → finish

# Initialize Groq LLM
llm = ChatGroq(model="llama-3.1-8b-instant", temperature=0.3)

# Reasoning Node (decide next step)
def reasoning_node(state: AgentState) -> AgentState:
    """The ReAct reasoning step — decide what to do next."""
    prompt = f"""
You are a smart AI assistant that decides how to handle a user query.

Query: {state.query}

If the query asks about:
- weather → respond with "WeatherAPI"
- knowledge / AI / ML / general info → respond with "VectorStoreSearch"

Otherwise → respond with "answer_directly"
    """

    decision = llm.invoke(prompt)
    tool_choice = decision.content.strip()

    # Update reasoning trace
    state.reasoning += f"\nModel decided to use tool: {tool_choice}"

    if "WeatherAPI" in tool_choice:
        state.next_action = "use_weather"
    elif "VectorStoreSearch" in tool_choice:
        state.next_action = "use_vector"
    else:
        state.next_action = "answer_directly"

    return state

In [67]:
def vector_tool_node(state: AgentState) -> AgentState:
    """Execute VectorStoreSearch tool."""
    output = search_vector_store(state.query)
    state.tool_output = output
    state.final_answer = output
    state.next_action = "finish"
    return state


def weather_tool_node(state: AgentState) -> AgentState:
    """Execute WeatherAPI tool."""
    output = get_weather(state.query)
    state.tool_output = output
    state.final_answer = output
    state.next_action = "finish"
    return state


def direct_answer_node(state: AgentState) -> AgentState:
    """LLM answers directly without tools."""
    prompt = f"Answer the following query directly:\n{state.query}"
    response = llm.invoke(prompt)
    state.final_answer = response.content.strip()
    state.next_action = "finish"
    return state


In [68]:
# Create Graph
workflow = StateGraph(AgentState)


# Add nodes
workflow.add_node("reasoning", reasoning_node)
workflow.add_node("vector_tool", vector_tool_node)
workflow.add_node("weather_tool", weather_tool_node)
workflow.add_node("direct_answer", direct_answer_node)

# Define edges (ReAct flow)
workflow.set_entry_point("reasoning")
workflow.add_conditional_edges(
    "reasoning",
    lambda s: s.next_action,
    {
        "use_vector": "vector_tool",
        "use_weather": "weather_tool",
        "answer_directly": "direct_answer",
    },
)
workflow.add_edge("vector_tool", END)
workflow.add_edge("weather_tool", END)
workflow.add_edge("direct_answer", END)

# Compile graph
react_agent = workflow.compile()


In [ ]:
queries = input()

while queries != "exit":
    print(f"\n🧩 Query: {queries}")
    result = react_agent.invoke(AgentState(query=queries))
    print("🤖 Final Answer:", result["final_answer"])


weather in naran

🧩 Query: weather in naran
🤖 Final Answer: OpenWeatherMap API key not found. Please add it to your environment variables.

🧩 Query: weather in naran
🤖 Final Answer: OpenWeatherMap API key not found. Please add it to your environment variables.

🧩 Query: weather in naran
🤖 Final Answer: OpenWeatherMap API key not found. Please add it to your environment variables.

🧩 Query: weather in naran
🤖 Final Answer: OpenWeatherMap API key not found. Please add it to your environment variables.

🧩 Query: weather in naran
🤖 Final Answer: OpenWeatherMap API key not found. Please add it to your environment variables.

🧩 Query: weather in naran
🤖 Final Answer: OpenWeatherMap API key not found. Please add it to your environment variables.

🧩 Query: weather in naran
🤖 Final Answer: OpenWeatherMap API key not found. Please add it to your environment variables.

🧩 Query: weather in naran
🤖 Final Answer: OpenWeatherMap API key not found. Please add it to your environment variables.

🧩 Quer